In [60]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
import pyautogui
import json
import os
import pyodbc
import pandas as pd

In [61]:
def GetConfig():
    global server
    global port
    global database
    global username
    global password

    config   = json.loads(open('config.json').read())
    server   = config['server']
    port     = config['port']
    database = config['database']
    username = config['username']
    password = config['password']

In [62]:
def linkedinconfig():
    global email
    global psswrd
    global keywords
    global location

    config = json.loads(open('linkedinconfig.json').read())
    email = config['user']
    psswrd = config['psswrd']
    keywords = config['keywords']
    location = config['location']

In [63]:
def connection():
    os.system('cls')
    print("[INFO] Abrindo conexão com banco de dados...")
    global CURSOR
    global CNXN
    while True:
        try:
            CNXN = pyodbc.connect(
                'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ',' +
                port + ';DATABASE=' +
                database + ';UID=' +
                username + ';PWD=' + password)
            CURSOR = CNXN.cursor()
            os.system('cls')
            print("[INFO] Conexão realizada com sucesso.")
            break
        except pyodbc.Error as ex:
            sleep(5)

In [64]:
def db_close():
    CURSOR.close()
    CNXN.close()

In [65]:
def fetch(sql):
    return pd.read_sql_query(sql, CNXN)

In [66]:
def insert(conteudo, cursor):
    sql = "INSERT INTO LinkedinVagas (CodVaga, Cargo, Localização, Conteudo) VALUES (?, ?, ?, ?, GetDate())"
    
    cursor.executemany(sql, conteudo)

In [67]:
def scroll(driver):
    SCROLL_PAUSE_TIME = 0.5

# Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [68]:
#CURSOR, CNXN = None, None

In [69]:
#GetConfig()
#connection()

In [70]:
linkedinconfig()

#CRIANDO DATAFRAME PARA ARMAZENAR OS DADOS

#ENTRANDO NA URL 
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get("https://www.linkedin.com/login/pt?session")

time.sleep(5)

C:\Users\Yan\AppData\Local\Temp\ipykernel_14272\2833837589.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [71]:
#LOGANDO NA CONTA
driver.find_element(By.ID, "username").send_keys(email)
driver.find_element(By.ID, "password").send_keys(psswrd)
time.sleep(0.5)
pyautogui.hotkey('enter')
time.sleep(30)

#ABRINDO A ABA DE VAGAS
driver.get('https://www.linkedin.com/jobs/')
time.sleep(15)
driver.find_element(By.ID, "keyword-typeahead-instance-ember24").click()
pyautogui.typewrite(keywords, interval=0.15)
pyautogui.hotkey('tab')
time.sleep(0.5)
pyautogui.typewrite(location, interval=0.15)
pyautogui.hotkey('enter')
time.sleep(10)
#COLETANDO AS INFORMAÇÕES
scroll(driver)      
time.sleep(5)
jobs_list = driver.find_elements(By.CLASS_NAME, 'scaffold-layout__list-container')
for job in jobs_list:
    print(job.text)
    time.sleep(2)

Biostatistician 1 ( Office/Home-Based, LATAM) - IQVIA Biotech
IQVIA
São Paulo, São Paulo, Brasil
Presencial
Você tem um selo de competência preferencial
Há 2 horas
Nas últimas 24 horas
Data Analytics & AI Solutions - REMOTE
UL
Brasil
Remoto
Você tem um selo de competência preferencial
Há 1 dia
24 candidaturas
Data Analytics & AI Solutions - REMOTE
UL
São Paulo, São Paulo, Brasil
Remoto
Você tem um selo de competência preferencial
Há 1 dia
17 candidaturas
Data Analytics & AI Solutions - REMOTE
UL
São Paulo, São Paulo, Brasil
Remoto
Você tem um selo de competência preferencial
Há 1 dia
Cientista de Dados Jr
Fretebras
São Paulo, São Paulo, Brasil
Remoto
2 ex-estudantes trabalham aqui
Há 1 semana
Data Scientist - Machine Learning
MUSA
São Paulo, São Paulo, Brasil
Remoto
Você tem um selo de competência preferencial
Há 3 dias
Data Scientist
Topaz
Brasil
Remoto
Combina com seu perfil
Há 3 semanas


In [ ]:
#NEXT STEPS - V1
"""1.SCROLL JOB WINDOW TO GET ALL THE INFORMATION;
   2. GET THE COMPLETE INFORMATION BY CLICKING IN EACH JOB;
   3. JUMP TO ALL THE PAGES COLLECTING INFORMATION;
   4. STORE THE DATA INTO A SQL DATABASE WITH PYODBC."""